In [ ]:
import re
import string
import operator
import functools
import glob
import os
import matplotlib.colors as mc
import nltk
import numpy as np
import textstat
import unicodedata
import pandas as pd

import textstat
from lexicalrichness import LexicalRichness
from collections import Counter

from nltk.corpus import stopwords
from nltk.stem import SnowballStemmer
from nltk.tokenize import word_tokenize
from nltk.tag import pos_tag, map_tag
from nltk.tokenize import sent_tokenize

 # Feature Selection
from sklearn.feature_selection import VarianceThreshold

In [ ]:
# The screen is cleaned
class ClearScreen():
    
    def __init__(self):
    
        self.clear = os.system('cls' if os.name=='nt' else 'clear')

In [ ]:
def countFiles(path, extension="*.txt"): 
    
    # Construir el patrón de búsqueda
    patron = os.path.join(path, extension)
    
    # Obtener la lista de archivos que coinciden con el patrón
    archivos = glob.glob(patron)
    
    return len(archivos) 

In [ ]:
class Segmentation:
    
    def __init__(self, text):
        
        self.text = text
        
    def sentSegmentation(self):
        
        return sent_tokenize(self.text)
    
    def paraSegmentation(self):
        
        texto = self.text.split("\n\n")
        
        return list(filter(bool, texto))      

In [ ]:
file = open(os.path.join(os.getcwd(), "corpus", "suspicious-document00013.txt"), encoding= "utf-8-sig")

text = file.read()
        
file.close()

In [ ]:
Seg = Segmentation(text)

sentSeg = Seg.sentSegmentation()

paraSeg = Seg.paraSegmentation()

In [ ]:
print(len(sentSeg))
print(len(paraSeg))

In [ ]:
for i, sentence in enumerate(sentSeg):
    print(f"Oracion: {i} --> Texto: {sentence}")

In [ ]:
for j, paragraph in enumerate(paraSeg):
    print(f"Parrafo: {j} --> Texto: {paragraph}")

In [ ]:
def getfleshReadingEase(text):
    
    fleshReadingEase = 0.0
    
    fleshReadingEase = textstat.flesch_reading_ease(text)
    
    return fleshReadingEase


def gettypeToken(text):    
    tam = len(text)
    if tam <= 0:        
        text = 0    
    else:
        try:
            text = LexicalRichness(text)    
            text = text.ttr
        except ZeroDivisionError:
            text = 0    
    return text

In [ ]:
datos = []

for i, value in enumerate(sentSeg):
    
    datos.append([i, value, getfleshReadingEase(value), gettypeToken(value)])

In [ ]:
df = pd.DataFrame(datos, columns=["index", "text", "fleshReadingEase", "typeToken"])

In [ ]:
df

In [ ]:
data = []

for j, value in enumerate(paragraph):
    
    data.append([i, value, getfleshReadingEase(value), gettypeToken(value)])

In [ ]:
df2 = pd.DataFrame(data, columns=["index", "text", "fleshReadingEase", "typeToken"])

df2